## Вопросы из тестового задания

### Cнизились продажи доступа к базе резюме
В первую очередь хотелось бы на сколько и за какой период упали продажи, а далее свериться с планом. Если план составлен корректно, с учетом сезонности и других особенностей бизнеса, то я бы сделал срезы по отраслям, сравнил бы их с предыдущими показателями, и на основе анализа выдвинул бы свои предположения

### Уменьшилось количество откликов соискателей на вакансии с вахтовой работой
Сделать срез по типу работы "Вахта", определить в каких регионах снижение прозоишло, воспользоваться внешними источниками для определения сезоности тех или иных работ в конкретных регионах, например, на севере лес заготавливают зимой и тд

### Выросла цена привлечения трафика на вакансии синих воротничков
Опять же нужно понять в каком секторе произошло повышение цен. Скорее всего причина в ужесточении законадательства по приеме на работу иностранных граждан, ввиду чего количество соискателей значительно уменьшилось, а вакансий нет. Я бы провел сравнительный анализ количества вакансий и соискателей за аналогичных период прошлых лет, на основе чего сделал бы вывод.

### Недовыполнен план продаж рекламы  
Здесь бы я провел параллель с вопросом "Cнизились продажи доступа к базе резюме". Корректно ли составлен план? Произошел ли отток клиентов по справнению с прошлым периодом, сколько новых клиентов к нам пришло? На основе анализа, можно говорить о работе отдела продажи и выдвинуть предполодения

### Cнизились оценки NPS и TRI*M от работодателей
В каких секторах? Нужно сделать срез данных по деятельности компаний, понимать где оценки пошли вниз, посмотреть динамику оценнок, какие факторы влияют на оценку? Что не устраивает клиента?

## Задание SQL

Для выполнения задания я создал импровизированную БД для удобство и наглядности запросов

## Загрузим Библиотеки

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('example.db')
def select(sql):
    return pd.read_sql(sql,conn)

## Создадим "произвольную" таблицу и загрузим в БД

In [2]:
negotation_topic = pd.DataFrame({'resume_id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,
                                              17,18,19,20,21,22,23,24,25,26,27,28,29,30],
                                'vacancy_id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                              18,19,20,21,22,23,24,25,26,27,28,29,30],
                                'state' : ['response','response','invitation','response','response','invitation',
                                          'invitation','invitation','invitation','invitation','response','invitation',
                                          'invitation','invitation','response','invitation','invitation','response',
                                          'invitation','response','response','response','response','response',
                                          'response','response','invitation','response','response','invitation']
                                })
negotation_topic.head()

,resume_id,vacancy_id,state
0,1,1,response
1,2,2,response
2,3,3,invitation
3,4,4,response
4,5,5,response


In [3]:
negotation_topic.to_sql('negotation_topic',conn,if_exists='replace', index = False)

### Класс, таблица создана, по аналогии созданим таблицу resume

In [4]:
resume = pd.DataFrame({'resume_id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,
                                    17,18,19,20,21,22,23,24,25,26,27,28,29,30],
                       'position' : ['водитель','грузчик','водитель','инженер','строитель','электрик',
                                     'строитель','сантехник','электрик','сантехник','охранник','сантехник',
                                     'сантехник','строитель','электрик','электрик','инженер','электрик',
                                     'строитель','инженер','грузчик','сантехник','электрик','электрик','электрик',
                                     'охранник','инженер','охранник','строитель','электрик']
                      })
resume.head()

,resume_id,position
0,1,водитель
1,2,грузчик
2,3,водитель
3,4,инженер
4,5,строитель


In [5]:
resume.to_sql('resume',conn,if_exists='replace',index=False)

## Проверим, корректно ли загрузились наши таблицы в базу

In [6]:
sql = '''select* from negotation_topic limit 5 '''

In [7]:
select(sql)

,resume_id,vacancy_id,state
0,1,1,response
1,2,2,response
2,3,3,invitation
3,4,4,response
4,5,5,response


In [8]:
sql = '''select * from resume limit 5'''

In [9]:
select(sql)

,resume_id,position
0,1,водитель
1,2,грузчик
2,3,водитель
3,4,инженер
4,5,строитель


### Супер, таблицы загруженны в базу!

## Выполним запросы SQL согласно заданию из моей БД
- Кого чаще приглашают на собеседование
- Кого реже приглашают на собеседование

### Кого чаще приглашают на собеседование

In [10]:
sql = ''' SELECT position,state, COUNT(state) AS invitation_count
          FROM resume 
          JOIN negotation_topic ON negotation_topic.resume_id = resume.resume_id
          WHERE state = 'invitation'
          GROUP BY state,position
          ORDER BY invitation_count DESC
          '''

In [11]:
select(sql)

,position,state,invitation_count
0,сантехник,invitation,4
1,электрик,invitation,4
2,строитель,invitation,3
3,инженер,invitation,2
4,водитель,invitation,1


### Сантехники и электрики самые полупярные специалисты - их чаще всего зовут на собеседование

### Кого реже приглашают на собеседование

In [12]:
sql = ''' SELECT position,state, COUNT(state) AS response_count
          FROM resume 
          JOIN negotation_topic ON negotation_topic.resume_id = resume.resume_id
          WHERE state = 'response'
          GROUP BY state,position
          ORDER BY response_count DESC
          '''

In [13]:
select(sql)

,position,state,response_count
0,электрик,response,5
1,охранник,response,3
2,грузчик,response,2
3,инженер,response,2
4,строитель,response,2
5,водитель,response,1
6,сантехник,response,1


### Не смотря на то, что электриков часто зовут на собеседование, им также часто и отказывают. Можно сделать вывод, что электриков много, а вакансий , видимо, нет. Абсолютно не востребованными являются охранники грузчики  - их вообще не зовут на собеседования... А вот сантехников явно не хватает - всего один отказ против 4 приглашений